In [1]:
import os

In [2]:
%pwd

'd:\\Scaler Course\\OLACaseStudy\\Ola-driver-churn\\research'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransormationConfig:
    root_dir: Path
    data_path: Path
    ohencoder_path: Path



In [5]:
from src.OLAChurnPred.constants import *
from src.OLAChurnPred.utils.common import read_yaml, create_directories
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransormationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransormationConfig(
            root_dir=config.root_dir,
            data_path = config.data_path,
            ohencoder_path=config.ohencoder_path
        )

        return data_transformation_config


In [6]:
import os
from src.OLAChurnPred import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from src.OLAChurnPred.utils.data_transformation_utils import *
from src.OLAChurnPred.utils.common import save_bin
from sklearn.preprocessing import OneHotEncoder

In [7]:
class DataTransormation:
    def __init__(self, config: DataTransormationConfig):
        self.config = config

    def transform(self):
        try:
            data = pd.read_csv(self.config.data_path)
            data = change_data_types(data)
            data = missing_value_imputation(data)
            data = group_transform_data(data)
            data = OHEncoding(data, self.config.ohencoder_path)
            return data
        except Exception as e:
            raise e
        
    def train_test_splitting(self):
        data = self.transform()
        train,test = train_test_split(data, test_size=0.20)

        train.to_csv(os.path.join(self.config.root_dir,"train.csv"),index=False)
        test.to_csv(os.path.join(self.config.root_dir,"test.csv"),index=False)
        logger.info("Train and test files created")
        print(train.shape, test.shape)    




In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransormation(config=data_transformation_config) 
    data_transformation.train_test_splitting()
except Exception as e:
    raise e

[2024-12-12 22:20:27,032: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-12 22:20:27,032: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-12 22:20:27,043: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-12 22:20:27,044: INFO: common: created directory at: artifacts]
[2024-12-12 22:20:27,045: INFO: common: created directory at: artifacts/data_transformation]
[2024-12-12 22:20:27,656: INFO: 3681127577: Train and test files created]
(1904, 39) (477, 39)


d:\Scaler Course\OLACaseStudy\Ola-driver-churn\src\OLAChurnPred\utils\data_transformation_utils.py:49: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  df1['Age'] = list(df.groupby('Driver_ID',axis=0).max('MMM-YY')['Age'])
d:\Scaler Course\OLACaseStudy\Ola-driver-churn\src\OLAChurnPred\utils\data_transformation_utils.py:56: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  df1['Total_Business_Value'] = list(df.groupby('Driver_ID',axis=0).sum('Total Business Value')['Total Business Value'])


In [9]:
df1 = pd.read_csv('D:\\Scaler Course\\OLACaseStudy\\Ola-driver-churn\\artifacts\\data_transformation\\test.csv')
df1.head()

,Age,Gender,Education,Income,Joining_Designation,Grade,Total_Business_Value,Last_Quarterly_Rating,Quarterly_Rating_Increased,Salary_Increased,...,C27,C28,C29,C3,C4,C5,C6,C7,C8,C9
0,34.0,0.0,2,54056,1,2,7563910,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,36.0,0.0,1,107149,3,3,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,29.0,1.0,0,39635,2,2,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,32.0,1.0,1,53181,1,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,29.0,0.0,2,50022,2,2,521860,1,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
